# DESARROLLO DE PROYECTOS Y PRODUCTOS DE DATOS
# Tarea 1
## Autor: Jesús Berríos
## Prof. Alonso Astroza

## **Código Server + Respuestas 2.1 y 2.3**


---
Se importan las bibliotecas a utilizar.

In [1]:
import cv2
import cvlib as cv
from cvlib.object_detection import draw_bbox
import io
import uvicorn
import numpy as np
import nest_asyncio
from enum import Enum
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import StreamingResponse, Response
import os


---
## **2.1 Agregue al programa servidor la opción de consultar al endpoint /predict agregando como parámetro el nivel de confianza**
Al código servidor se ejecutan los siguientes cambios:

2.1.1. Se agrega input "conf" al endpoint predict, con un valor por defecto de 0.5. Adicionalmente, se toma el valor de confidence convirtiéndolo a float ya que la información viene como string en los parámetros de la URL.
```
@app.post("/predict") 
def prediction(model: Model, file: UploadFile = File(...), conf = 0.5):
    
    confidence_value = float(conf)
```

2.1.2 Se añade valor de confidence a la función detect_common_objects de OpenCV, la cual tiene como entrada la imagen a detectar, el modelo y el valor de confidence.
```
bbox, label, conf = cv.detect_common_objects(image, confidence = confidence_value, model=model)
```


---
## **2.3 En el programa servidor implemente un nuevo endpoint /countObjects para realizar un contador de un objeto en particular**
Al código servidor se ejecutan los siguientes cambios:

2.3.1. Se agrega endpoint countObjects el cual tiene como entrada el modelo, la imagen, el valor de confidence y el objeto que se desea contar. Este valor se sincroniza con el formato de URL del lado cliente.
```
@app.post("/countObjects") 
def prediction(model: Model, file: UploadFile = File(...), conf = 0.5, object =''):
```

2.3.2 Se agrega código que recorre las labels detectadas y si alguna de las labels coincide con el objeto solicitado por el cliente se aumenta el contador
```
bbox, label, conf = cv.detect_common_objects(image, confidence = confidence_value, model=model)

    cnt=0
    for i in label:
        if object in i:
            cnt+=1
```

2.3.3 Se retorna un response simple en la función con la cantidad de objetos que se solicitó contar desde el cliente. Se añade validación en la respuesta:
```
lista_objetos = ['person','bicycle','car','motorcycle','airplane','bus','train','truck','boat','traffic light','fire hydrant','stop sign','parking meter','bench','bird','cat','dog','horse','sheep','cow','elephant','bear','zebra','giraffe','backpack','umbrella','handbag','tie','suitcase','frisbee','skis','snowboard','sports ball','kite','baseball bat','baseball glove','skateboard','surfboard','tennis racket','bottle','wine glass','cup','fork','knife','spoon','bowl','banana','apple','sandwich','orange','broccoli','carrot','hot dog','pizza','donut','cake','chair','couch','potted plant','bed','dining table','toilet','tv','laptop','mouse','remote','keyboard','cell phone','microwave','oven','toaster','sink','refrigerator','book','clock','vase','scissors','teddy bear','hair drier','toothbrush']

if object in lista_objetos:
    respuesta = f'Cantidad_{object}={cnt}'
else:
    respuesta = 'Favor ingresar un objeto valido para su deteccion'

# Retornar la respuesta como un string con el dato solicitado
return Response(respuesta)
```

---
## **Código server**

Crear carpeta si no existe:

In [2]:
dir_name = "images_uploaded"
if not os.path.exists(dir_name):
    os.mkdir(dir_name)

### **Funcionalidad Server**

In [3]:
# Asignamos una instancia de la clase FastAPI a la variable "app".
# Interacturaremos con la API usando este elemento.
app = FastAPI(title='Implementando un modelo de Machine Learning usando FastAPI')

# Enlistamos los modelos disponibles usando Enum. Útil cuando tenemos opciones predefinidas.
class Model(str, Enum):
    yolov3tiny = "yolov3-tiny"
    yolov3 = "yolov3"


# Usando @app.get("/") definimos un método GET para el endpoint / (que sería como el "home").
@app.get("/")
def home():
    return "¡Felicitaciones! Tu API está funcionando según lo esperado. Anda ahora a http://localhost:8000/docs."


# Este endpoint maneja la lógica necesaria para detectar objetos.
# Requiere como entrada el modelo deseado y la imagen.
# Se añade valor de confidence a la función
@app.post("/predict") 
def prediction(model: Model, file: UploadFile = File(...), conf = 0.5):
    
    # 0. Se convierte valor de confidence a Float
    confidence_value = float(conf)

    # 1. Validar el archivo de entrada
    filename = file.filename
    fileExtension = filename.split(".")[-1] in ("jpg", "jpeg", "png")
    if not fileExtension:
        raise HTTPException(status_code=415, detail="Tipo de archivo no soportado.")
    
    # 2. Transformar la imagen cruda a una imagen CV2
    # Leer la imagen como un stream de bytes
    image_stream = io.BytesIO(file.file.read())
    # Empezar el stream desde el principio (posicion cero)
    image_stream.seek(0)
    # Escribir el stream en un numpy array
    file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
    # Decodificar el numpy array como una imagen
    image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
    
    
    # 3. Correr el modelo de detección de objetos
    # Correr la detección de objetos
    bbox, label, conf = cv.detect_common_objects(image, confidence = confidence_value, model=model)
    # Crear una imagen que contenga las cajas delimitadoras y etiquetas
    output_image = draw_bbox(image, bbox, label, conf)
    # Guardarla en un directorio del server
    cv2.imwrite(f'images_uploaded/{filename}', output_image)
    
    # 4. Transmitir la respuesta de vuelta al cliente
    # Abrir la imagen para leerla en formato binario
    file_image = open(f'images_uploaded/{filename}', mode="rb")
    # Retornar la imagen como un stream usando un formato específico
    return StreamingResponse(file_image, media_type="image/jpeg")#,f'Confidence_used={confidence_value}']

# Este endpoint maneja la lógica necesaria para contar objetos.
# Requiere como entrada el modelo deseado, la imagen, nivel de confidence y objeto a detectar
@app.post("/countObjects") 
def prediction(model: Model, file: UploadFile = File(...), conf = 0.5, object =''):
    
    # 0. Se transforma el valor de confidence a float
    confidence_value = float(conf)

    # 1. Validar el archivo de entrada
    filename = file.filename
    fileExtension = filename.split(".")[-1] in ("jpg", "jpeg", "png")
    if not fileExtension:
        raise HTTPException(status_code=415, detail="Tipo de archivo no soportado.")
    
    # 2. Transformar la imagen cruda a una imagen CV2
    # Leer la imagen como un stream de bytes
    image_stream = io.BytesIO(file.file.read())
    # Empezar el stream desde el principio (posicion cero)
    image_stream.seek(0)
    # Escribir el stream en un numpy array
    file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
    # Decodificar el numpy array como una imagen
    image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
    
    # 3. Correr el modelo de detección de objetos
    # Correr la detección de objetos
    bbox, label, conf = cv.detect_common_objects(image, confidence = confidence_value, model=model)
    # Contar la cantidad de objetos requeridas por el cliente
    cnt=0
    for i in label:
        if object in i:
            cnt+=1
    
    # 4. Se valida si el objeto solicitado se puede detectar por el modelo
    lista_objetos = ['person','bicycle','car','motorcycle','airplane','bus','train','truck','boat','traffic light','fire hydrant','stop sign','parking meter','bench','bird','cat','dog','horse','sheep','cow','elephant','bear','zebra','giraffe','backpack','umbrella','handbag','tie','suitcase','frisbee','skis','snowboard','sports ball','kite','baseball bat','baseball glove','skateboard','surfboard','tennis racket','bottle','wine glass','cup','fork','knife','spoon','bowl','banana','apple','sandwich','orange','broccoli','carrot','hot dog','pizza','donut','cake','chair','couch','potted plant','bed','dining table','toilet','tv','laptop','mouse','remote','keyboard','cell phone','microwave','oven','toaster','sink','refrigerator','book','clock','vase','scissors','teddy bear','hair drier','toothbrush']
    if object in lista_objetos:
        respuesta = f'Cantidad_{object}={cnt}'
    else:
        respuesta = 'Favor ingresar un objeto valido para su deteccion'

    # Retornar la respuesta como un string con el dato solicitado
    return Response(respuesta)

### **Iniciar el server**

In [4]:
# Esto deja correr al servidor en un ambiente interactivo como un Jupyter notebook
nest_asyncio.apply()

# Donde se hospedará el servidor
host = "127.0.0.1"

# ¡Iniciemos el servidor!    
uvicorn.run(app, host=host, port=8000)

INFO:     Started server process [2666]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:52890 - "POST /predict?model=yolov3-tiny&conf=0.67 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52901 - "POST /predict?model=yolov3-tiny&conf=0.67 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52909 - "POST /predict?model=yolov3-tiny&conf=0.1 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52910 - "POST /predict?model=yolov3-tiny&conf=0.2 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52911 - "POST /predict?model=yolov3-tiny&conf=0.3 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52912 - "POST /predict?model=yolov3-tiny&conf=0.4 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52913 - "POST /predict?model=yolov3-tiny&conf=0.5 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52914 - "POST /predict?model=yolov3-tiny&conf=0.6 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52915 - "POST /predict?model=yolov3-tiny&conf=0.7 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52916 - "POST /predict?model=yolov3-tiny&conf=0.8 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52917 - "POST /predict?model=yolov3-tiny&conf=0.9 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52919 - "POST /predic

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2666]
